In [1]:
import torch.nn as nn
import torch
import numpy as np
import gym

import ipynb.fs.full.misc as misc
import ipynb.fs.full.breeder as br
import ipynb.fs.full.mutation_functions as mf
import ipynb.fs.full.selection_functions as sf
import ipynb.fs.full.crossover_functions as cf
from ipynb.fs.full.agent_class import Agent
import ipynb.fs.full.net_class as nm

In [2]:
class PongAgentNet(nm.NetModule):
    def __init__(self):
        super(PongAgentNet, self).__init__()

        self.num_outputs = 6

        #  define net structure here
        self.model = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(),            
            nn.MaxPool2d(2),           
            
            nn.Conv2d(16, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2), 
            
            nn.Conv2d(16, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        ).cuda()
        
        self.classifier = nn.Sequential(
            nn.Linear(26*20*16, 32),
            nn.Tanh(),            
            nn.Linear(32, self.num_outputs),
            nn.Tanh()
        ).cuda()
       

        self.parameters_count = self.count_parameters()

    def forward(self, inputs):
        c = self.model(inputs)
        
        c = c.view(c.size(0), -1)
        
        c = self.classifier(c)
        
        return torch.argmax(c, dim=1)

class PongAgent(Agent):
    def __init__(self, dna=None):
        super(PongAgent, self).__init__(dna, PongAgentNet())

    def step(self, observation):
        state = np.asarray(observation).transpose(2, 0, 1)        

        torch_state = misc.single_to_torch(state, cuda=self.net.is_cuda())        
        torch_action = self.net(torch_state)
        action = misc.single_from_torch(torch_action, cuda=self.net.is_cuda())

        self.add_experience(state, action)

        return action

    @staticmethod
    def produce_offspring(dna):
        return PongAgent(dna=dna)

misc.init_random(1337)

env = gym.make("Pong-v0")


population_size = 50
population = [PongAgent()] * population_size

breeder = br.Breeder(population,
                     env,
                     195.,
                     generations=200,
                     elite_percent=0.1,
                     mutation_rate=0.1,
                     selection_pressure=0.2,
                     selection_func=sf.tournament,
                     crossover_func=cf.none,
                     mutation_func=mf.sm_g_so,
                     render=False)

agent = breeder.evolve()


for _ in range(10):
    agent.evaluate(env, render=True, sleep=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Gen: 0 , Max fitness: -21.0 , Min fitness: -21.0 , Age: 0 , Mean fitness: -21.0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Gen: 1 , Max fitness: -21.0 , Min fitness: -21.0 , Age: 5 , Mean fitness: -21.0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


KeyboardInterrupt: 